<a href="https://colab.research.google.com/github/vansh123321/vansh/blob/master/Word_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In December 2019, a pneumonia outbreak was reported in Wuhan, China. On 31 December 2019, the outbreak was traced to a novel strain of coronavirus. In the next few months this disease spread across multiple countries and continents. As of 9 April 2020, there have been at least 88,538 confirmed deaths and more than 1,484,811 confirmed cases in the conronavirus pneumonia pandemic. The pandemic has resulted in travel restrictions and nationwide lockdowns in several countries. Since, this is an issue of great concern worldwide, we have made a project associated with this topic. This project is a semantic analysis on COVID-19 research papers. We have taken many such research papers as our data set. It aims to help a user find other related words to keywords such as corona, virus, infection, etc. from the research papers. The program takes ‘a keyword’ and ‘the number of related words needed’ as input and display the required number of related words. This can help the user easily find out more about coronavirus and consequently spread awareness.


**Import Libraries** 

In [9]:
!pip install top2vec
import numpy as np 
import pandas as pd 
import json
import os
import ipywidgets as widgets
from IPython.display import clear_output, display
from top2vec import Top2Vec

In [10]:
!pip install kaggle

In [11]:
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"vanshjatana","key":"c38bd2c70d585ec85deaead606a1d1cd"}'}

In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

CORD-19-research-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
!kaggle datasets download -d dangelov/covid19top2vec

covid19top2vec.zip: Skipping, found more recently modified local copy (use --force to force download)


In [15]:
from zipfile import ZipFile
file_name = "CORD-19-research-challenge.zip"
with ZipFile(file_name,'r') as zp:
  zp.extractall()
  print('done')

done


In [16]:
from zipfile import ZipFile
file_name = "covid19top2vec.zip"
with ZipFile(file_name,'r') as zp:
  zp.extractall()
  print('done')

done


In [17]:
metadata_df = pd.read_csv("/content/metadata.csv")
metadata_df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,13,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [18]:
def preproccess_papers():

    dataset_dir = "/content/CORD-19-research-challenge/"
    comm_dir = dataset_dir+"comm_use_subset/comm_use_subset/"
    noncomm_dir = dataset_dir+"noncomm_use_subset/noncomm_use_subset/"
    custom_dir = dataset_dir+"custom_license/custom_license/"
    biorxiv_dir = dataset_dir+"biorxiv_medrxiv/biorxiv_medrxiv/"
    directories_to_process = [comm_dir,noncomm_dir, custom_dir, biorxiv_dir]

    papers_with_text = list(metadata_df[metadata_df.has_full_text==True].sha)

    paper_ids = []
    titles = []
    abstracts = []
    sections = []
    body_texts = []

    for directory in directories_to_process:

        filenames = os.listdir(directory)

        for filename in filenames:

          file = json.load(open(directory+filename, 'rb'))

          #check if file contains text
          if file["paper_id"] in papers_with_text:

            section = []
            text = []

            for bod in file["body_text"]:
              section.append(bod["section"])
              text.append(bod["text"])

            res_df = pd.DataFrame({"section":section, "text":text}).groupby("section")["text"].apply(' '.join).reset_index()

            for index, row in res_df.iterrows():

              # metadata
              paper_ids.append(file["paper_id"])

              if(len(file["abstract"])):
                abstracts.append(file["abstract"][0]["text"])
              else:
                abstracts.append("")

              titles.append(file["metadata"]["title"])

              # add section and text
              sections.append(row.section)
              body_texts.append(row.text)

    return pd.DataFrame({"id":paper_ids, "title": titles, "abstract": abstracts, "section": sections, "text": body_texts})

In [19]:
def filter_short(papers_df):
    papers_df["token_counts"] = papers_df["text"].str.split().map(len)
    papers_df = papers_df[papers_df.token_counts>200].reset_index(drop=True)
    papers_df.drop('token_counts', axis=1, inplace=True)
    
    return papers_df
   

In [20]:
top2vec = Top2Vec.load("/content/covid19_deep_learn_top2vec")

In [21]:
papers_df = pd.read_feather("/content/covid19_papers_processed.feather")

SyntaxError: ignored

In [23]:
keywords_select_sw = widgets.Label('Enter Keyword')
display(keywords_select_sw)


keywords_input_sw = widgets.Text()
display(keywords_input_sw)



doc_num_select_sw = widgets.Label('Choose number of words: ')
display(doc_num_select_sw)

doc_num_input_sw = widgets.Text(value='5')
display(doc_num_input_sw)

def display_similar_words(*args):
    
    clear_output()
    display(keywords_select_sw)
    display(keywords_input_sw)
    display(doc_num_select_sw)
    display(doc_num_input_sw)
    display(sim_word_btn_sw)
    
    try:            
        words, word_scores = top2vec.similar_words(keywords=keywords_input_sw.value.split(), num_words=int(doc_num_input_sw.value))
        for word, score in zip(words, word_scores):
            print(f"{word} {score}")

    except Exception as e:
        print(e)
        
sim_word_btn_sw = widgets.Button(description="Show similar words")
display(sim_word_btn_sw)
sim_word_btn_sw.on_click(display_similar_words)

Label(value='Enter Keyword')

Text(value='virus')

Label(value='Choose number of words: ')

Text(value='5')

Button(description='Show similar words', style=ButtonStyle())

viruses 0.8135
viral 0.7248
infection 0.6914
human 0.6628
influenza 0.6403


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
